### THE GENERATED CODE BY LLM

In [2]:
from z3 import *

def compute_tax(category, subcategory, quantity, alcohol_content=None):
    """
    計算菸酒稅的應納稅額
    參數:
      category: '菸' 或 '酒'
      subcategory: 
         若 category 為 '菸'，則必須為 1~8（依題目定義）
           1. 紙菸（新制，106年6月11日以後）：每千支 1590元
           2. 菸絲（新制）：每公斤 1590元
           3. 雪茄（新制）：每公斤 1590元
           4. 其他菸品（新制）：每公斤 1590元  （※行政院112.4.1規定，取每公斤或每千支之較高者，這裡簡化取 1590元）
           5. 紙菸（舊制，106年6月11日以前）：每千支 590元   【依 菸酒稅法第7條】
           6. 菸絲（舊制）：每公斤 590元                         【依 菸酒稅法第7條】
           7. 雪茄（舊制）：每公斤 590元                         【依 菸酒稅法第7條】
           8. 其他菸品（舊制）：每公斤 590元                       【依 菸酒稅法第7條】
         若 category 為 '酒'，則必須為 1~10（依題目定義）
           1. 釀造酒類：啤酒，每公升 26元                        【依 酒類規定】
           2. 釀造酒類：其他釀造酒，每公升按酒精成分每度 7元       【依 酒類規定】
           3. 蒸餾酒類：每公升按酒精成分每度 2.5元                  【依 酒類規定】
           4. 再製酒類：酒精成分超過20%者，每公升 185元             【依 酒類規定】
           5. 再製酒類：酒精成分20%或以下者，每公升按酒精成分每度 7元 【依 酒類規定】
           6. 料理酒類：每公升 9元                                【依 酒類規定】
           7. 料理酒類：97年5月15日以前出廠者，每公升 22元          【依 酒類規定】
           8. 其他酒類：每公升按酒精成分每度 7元                   【依 酒類規定】
           9. 酒精：每公升 15元                                  【依 酒類規定】
          10. 酒精：97年5月15日以前出廠者，每公升 11元            【依 酒類規定】
      quantity: 當月份出廠應稅數量（單位依子類別而定：菸類為「支」或「公斤」，酒類為「公升」）
      alcohol_content: 酒精成分（度數）；若為酒類且有依酒精成分計算，該參數必須提供
  
    return:
      應納稅額（元）
    """
    # 建立 Z3 求解器
    s = Solver()
    tax = Real('tax')  # 最終稅額

    if category == '菸':
        # 菸類另外徵收健康福利捐：
        # 紙菸、菸絲、雪茄、其他菸品均為每單位（千支或公斤）徵收 1000元
        if subcategory in [1, 5]:  # 紙菸：依生產日期區分新制/舊制
            # 單位：每千支
            # 新制（subcat 1）：基本稅率 1590元【每千支徵收1,590元】
            # 舊制（subcat 5）：基本稅率 590元  【依 菸酒稅法第7條】
            base_rate = 1590 if subcategory == 1 else 590
            health_rate = 1000  # 【菸品健康福利捐規定】
            unit_divisor = 1000  # 計算單位為千支
        elif subcategory in [2, 6]:  # 菸絲：單位為公斤
            base_rate = 1590 if subcategory == 2 else 590  # 【依 菸酒稅法第7條】
            health_rate = 1000  # 【菸品健康福利捐規定】
            unit_divisor = 1
        elif subcategory in [3, 7]:  # 雪茄：單位為公斤
            base_rate = 1590 if subcategory == 3 else 590  # 【依 菸酒稅法第7條】
            health_rate = 1000
            unit_divisor = 1
        elif subcategory in [4, 8]:  # 其他菸品：單位為公斤
            base_rate = 1590 if subcategory == 4 else 590  # 【依 菸酒稅法第7條】
            health_rate = 1000
            unit_divisor = 1
        else:
            raise ValueError("菸類子類別需為1至8中的數字。")
        
        # 計算公式： (基本稅率 + 健康福利捐稅率) * (出廠數量 / 單位)
        tax_expr = (base_rate + health_rate) * (quantity / unit_divisor)
        s.add(tax == tax_expr)
    
    elif category == '酒':
        # 酒類：必須提供 alcohol_content（酒精成分，單位：度）
        if alcohol_content is None:
            raise ValueError("酒類計算時，必須提供酒精成分(度數)")
        # 使用 Z3 的 If 條件式依子類別選擇對應的稅率：
        rate = If(subcategory == 1, 26,                      # 釀造酒類：啤酒 每公升26元 【依 酒類規定】
              If(subcategory == 2, 7 * alcohol_content,      # 釀造酒類：其他釀造酒 每公升按酒精成分每度7元 【依 酒類規定】
              If(subcategory == 3, 2.5 * alcohol_content,      # 蒸餾酒類：每公升按酒精成分每度2.5元 【依 酒類規定】
              If(subcategory == 4, 185,                        # 再製酒類：酒精成分超過20%者 每公升185元 【依 酒類規定】
              If(subcategory == 5, 7 * alcohol_content,        # 再製酒類：酒精成分20%或以下者 每公升按酒精成分每度7元 【依 酒類規定】
              If(subcategory == 6, 9,                          # 料理酒類：每公升9元 【依 酒類規定】
              If(subcategory == 7, 22,                         # 料理酒類：97年5月15日以前出廠者 每公升22元 【依 酒類規定】
              If(subcategory == 8, 7 * alcohol_content,        # 其他酒類：每公升按酒精成分每度7元 【依 酒類規定】
              If(subcategory == 9, 15,                         # 酒精：每公升15元 【依 酒類規定】
              If(subcategory == 10, 11,                        # 酒精：97年5月15日以前出廠者 每公升11元 【依 酒類規定】
              0))))))))))
        # 計算公式： 稅率 * 當月份出廠數量(公升)
        tax_expr = rate * quantity
        s.add(tax == tax_expr)
    
    else:
        raise ValueError("類別必須為 '菸' 或 '酒'")
    
    # 求解
    if s.check() == sat:
        m = s.model()
        return m.evaluate(tax)
    else:
        return None

#---------------------------------------------
# 測試範例

if __name__ == "__main__":
    # 範例1：菸類，假設使用新制 紙菸 (子類別 1)，當月份出廠 2000 支
    tax_tobacco = compute_tax(category='菸', subcategory=1, quantity=2000)
    print("菸類稅額 (新制紙菸, 2000支):", tax_tobacco)
    
    # 範例2：菸類，舊制 菸絲 (子類別 6)，當月份出廠 50 公斤
    tax_tobacco_old = compute_tax(category='菸', subcategory=6, quantity=50)
    print("菸類稅額 (舊制菸絲, 50公斤):", tax_tobacco_old)
    
    # 範例3：酒類，啤酒 (子類別 1)，當月份出廠 100 公升 (啤酒不需要酒精成分)
    tax_alcohol_beer = compute_tax(category='酒', subcategory=1, quantity=100, alcohol_content=0)
    print("酒類稅額 (啤酒, 100公升):", tax_alcohol_beer)
    
    # 範例4：酒類，其他釀造酒 (子類別 2)，酒精成分 5 度，當月份出廠 80 公升
    tax_alcohol_other = compute_tax(category='酒', subcategory=2, quantity=80, alcohol_content=5)
    print("酒類稅額 (其他釀造酒, 80公升, 酒精5度):", tax_alcohol_other)


菸類稅額 (新制紙菸, 2000支): 5180
菸類稅額 (舊制菸絲, 50公斤): 79500
酒類稅額 (啤酒, 100公升): 2600
酒類稅額 (其他釀造酒, 80公升, 酒精5度): 2800


### Selenium Test

In [17]:
import random
import time
import re
from decimal import Decimal, ROUND_HALF_UP

from z3 import Real, Solver, sat, RealVal

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import (
    TimeoutException,
    ElementClickInterceptedException,
    StaleElementReferenceException,
)

URL = "https://www.etax.nat.gov.tw/etwmain/etw158w/75"


# ------------------ Z3 稅額計算（精確有理數） ------------------
def _rat_to_decimal(z3_rat) -> Decimal:
    num = z3_rat.numerator_as_long()
    den = z3_rat.denominator_as_long()
    return Decimal(num) / Decimal(den)


def compute_tax(category, subcategory, quantity, alcohol_content=None):
    """
    計算菸酒稅（整數，ROUND_HALF_UP）
    category: '菸' or '酒'
    subcategory: 菸 1~8 / 酒 1~10
    """
    s = Solver()
    tax = Real("tax")

    if category == "菸":
        if subcategory in (1, 5):  # 紙菸：每千支
            base_rate = 1590 if subcategory == 1 else 590
            unit_divisor = 1000
        elif subcategory in (2, 6):  # 菸絲：每公斤
            base_rate = 1590 if subcategory == 2 else 590
            unit_divisor = 1
        elif subcategory in (3, 7):  # 雪茄：每公斤
            base_rate = 1590 if subcategory == 3 else 590
            unit_divisor = 1
        elif subcategory in (4, 8):  # 其他菸品：每公斤（此處簡化）
            base_rate = 1590 if subcategory == 4 else 590
            unit_divisor = 1
        else:
            raise ValueError("菸類子類別需為 1~8")

        expr = RealVal(str(base_rate)) * RealVal(str(quantity)) / RealVal(str(unit_divisor))
        s.add(tax == expr)

    elif category == "酒":
        if alcohol_content is None:
            alcohol_content = 1

        alc = RealVal(str(alcohol_content))

        # 依你提供的定義直接分支（避免深層 If 括號/縮排爆炸）
        if subcategory == 1:
            rate = RealVal("26")
        elif subcategory == 2:
            rate = RealVal("7") * alc
        elif subcategory == 3:
            rate = RealVal("5") * alc / RealVal("2")  # 2.5 = 5/2
        elif subcategory == 4:
            rate = RealVal("185")
        elif subcategory == 5:
            rate = RealVal("7") * alc
        elif subcategory == 6:
            rate = RealVal("9")
        elif subcategory == 7:
            rate = RealVal("22")
        elif subcategory == 8:
            rate = RealVal("7") * alc
        elif subcategory == 9:
            rate = RealVal("15")
        elif subcategory == 10:
            rate = RealVal("11")
        else:
            raise ValueError("酒類子類別需為 1~10")

        expr = rate * RealVal(str(quantity))
        s.add(tax == expr)

    else:
        raise ValueError("類別必須為 '菸' 或 '酒'")

    if s.check() != sat:
        return None

    m = s.model()
    dec = _rat_to_decimal(m.evaluate(tax))
    return int(dec.to_integral_value(rounding=ROUND_HALF_UP))


# ------------------ Selenium：逐字輸入（維持你的速度） ------------------
def human_type(element, text, delay=0.1):
    for ch in str(text):
        element.send_keys(ch)
        time.sleep(delay)
    element.send_keys(Keys.TAB)


# ------------------ Selenium：工具 ------------------
def safe_clear(driver, el):
    try:
        el.clear()
    except Exception:
        driver.execute_script("arguments[0].value='';", el)


def dismiss_gsc_overlay(driver):
    """
    站內搜尋的 overlay 有時會擋住點擊；能關就關，不能就隱藏
    """
    try:
        close_btns = driver.find_elements(By.CSS_SELECTOR, ".gsc-results-close-btn")
        for b in close_btns:
            if b.is_displayed() and b.is_enabled():
                try:
                    b.click()
                    time.sleep(0.05)
                except Exception:
                    pass

        try:
            driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
            time.sleep(0.05)
        except Exception:
            pass

        overlays = driver.find_elements(By.CSS_SELECTOR, ".gsc-results-wrapper-overlay")
        for ov in overlays:
            if ov.is_displayed():
                driver.execute_script(
                    "arguments[0].style.display='none'; arguments[0].style.pointerEvents='none';",
                    ov
                )
    except Exception:
        pass


def safe_click(driver, el):
    dismiss_gsc_overlay(driver)
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    time.sleep(0.1)
    for _ in range(3):
        try:
            el.click()
            return
        except (ElementClickInterceptedException, StaleElementReferenceException):
            dismiss_gsc_overlay(driver)
            try:
                driver.execute_script("arguments[0].click();", el)
                return
            except Exception:
                time.sleep(0.2)
    raise RuntimeError("click 失敗（可能被遮罩層/動畫擋住）")


def _parse_int_attr(x):
    if x is None:
        return None
    x = str(x).strip()
    return int(x) if re.fullmatch(r"-?\d+", x) else None


def get_int_constraints(el, default_min=1, default_max=100):
    min_attr = _parse_int_attr(el.get_attribute("min"))
    max_attr = _parse_int_attr(el.get_attribute("max"))
    maxlength = _parse_int_attr(el.get_attribute("maxlength"))

    min_v = min_attr if min_attr is not None else default_min
    max_v = max_attr if max_attr is not None else default_max

    if maxlength is not None and maxlength > 0:
        max_by_len = (10 ** maxlength) - 1
        max_v = min(max_v, max_by_len)

    if max_v < min_v:
        max_v = min_v
    return min_v, max_v


def parse_int_from_text(s):
    if not s:
        return None
    s = str(s).strip().replace(",", "")
    m = re.search(r"-?\d+(?:\.\d+)?", s)
    if not m:
        return None
    try:
        return int(Decimal(m.group(0)).to_integral_value(rounding=ROUND_HALF_UP))
    except Exception:
        return None


def valid_option_indices(sel: Select):
    """
    不依賴 value，只排除 index0/請選擇/disabled
    """
    idxs = []
    for i, opt in enumerate(sel.options):
        if i == 0:
            continue
        if not opt.is_enabled():
            continue
        t = (opt.text or "").strip()
        if not t:
            continue
        if "請選擇" in t:
            continue
        idxs.append(i)
    return idxs


def wait_select_ready(driver, wait: WebDriverWait, select_id: str, timeout=12):
    """
    等到 select 有至少 1 個有效 option（排除 placeholder）
    """
    end = time.time() + timeout
    while time.time() < end:
        try:
            sel = Select(driver.find_element(By.ID, select_id))
            if len(valid_option_indices(sel)) >= 1:
                return
        except Exception:
            pass
        time.sleep(0.1)
    raise TimeoutException(f"{select_id} 等待 options ready 超時")


def trigger_change(driver, select_el):
    """
    某些頁面只靠 select_by_index 不一定觸發到 JS listener，補發 change event
    """
    try:
        driver.execute_script(
            "arguments[0].dispatchEvent(new Event('change', {bubbles:true}));",
            select_el
        )
    except Exception:
        pass


def find_degree_input(driver):
    """
    degree 欄位可能動態出現，用多策略找
    """
    candidates = [
        (By.ID, "degree"),
        (By.NAME, "degree"),
        (By.CSS_SELECTOR, "input#degree"),
        (By.CSS_SELECTOR, "input[name='degree']"),
        (By.CSS_SELECTOR, "input[id*='degree' i]"),
        (By.XPATH, "//label[contains(normalize-space(.), '酒精')]/following::input[1]"),
        (By.XPATH, "//input[contains(@placeholder,'酒精') or contains(@aria-label,'酒精')]"),
        (By.XPATH, "//input[contains(@id,'deg') or contains(@name,'deg')]"),
    ]
    for by, sel in candidates:
        try:
            el = driver.find_element(by, sel)
            if el.is_displayed():
                return el
        except Exception:
            pass
    return None


def choose_alcohol_content_for_subcat(subcat, degree_el=None):
    """
    依你給的定義生成合法酒精度，並跟 input 的 min/max 取交集避免違規
      2/3/8: 1..95
      4: >20 => 21..95
      5: <=20 => 1..20
      其他: 1
    """
    if subcat == 4:
        lb, ub = 21, 95
    elif subcat == 5:
        lb, ub = 1, 20
    elif subcat in (2, 3, 8):
        lb, ub = 1, 95
    else:
        lb, ub = 1, 1

    if degree_el is not None:
        dmin, dmax = get_int_constraints(degree_el, default_min=lb, default_max=ub)
        lb = max(lb, dmin)
        ub = min(ub, dmax)
        if ub < lb:
            # 交集崩掉保底
            ub = lb

    return random.randint(lb, ub)


def ensure_degree(driver, sel3_el, subcat, must_exist, retries=2):
    """
    確保 degree 出現並填好
    回傳 (ok, alcohol_content_used or None)
    """
    for _ in range(retries):
        time.sleep(0.3)
        el = find_degree_input(driver)
        if el is None:
            if must_exist:
                # 重觸發一次 change 再找
                trigger_change(driver, sel3_el)
                time.sleep(0.4)
                el = find_degree_input(driver)
                if el is None:
                    continue
            else:
                return True, None

        alcohol_content = choose_alcohol_content_for_subcat(subcat, el)
        try:
            safe_clear(driver, el)
            if el.is_enabled():
                human_type(el, str(alcohol_content), delay=0.1)
            else:
                driver.execute_script("arguments[0].value = arguments[1];", el, str(alcohol_content))
                try:
                    el.send_keys(Keys.TAB)
                except Exception:
                    pass
            return True, alcohol_content
        except Exception:
            trigger_change(driver, sel3_el)
            time.sleep(0.3)

    return False, None


def read_result_near_anchor(driver, anchor_id, timeout=10.0):
    """
    span.text-danger 可能不只一個，用 anchor 的 y 座標找最近那個
    """
    end = time.time() + timeout
    while time.time() < end:
        spans = [s for s in driver.find_elements(By.CSS_SELECTOR, "span.text-danger") if s.is_displayed()]
        cands = []
        for sp in spans:
            v = parse_int_from_text(sp.text)
            if v is not None:
                cands.append((sp, v))

        if cands:
            try:
                anchor = driver.find_element(By.ID, anchor_id)
            except Exception:
                return cands[0][1]

            try:
                idx = driver.execute_script(
                    """
                    const anchor = arguments[0];
                    const spans = arguments[1];
                    const ay = anchor.getBoundingClientRect().top;
                    let best = 0, bestd = 1e18;
                    for (let i=0; i<spans.length; i++) {
                      const r = spans[i].getBoundingClientRect();
                      const d = Math.abs(r.top - ay);
                      if (d < bestd) { bestd = d; best = i; }
                    }
                    return best;
                    """,
                    anchor, [c[0] for c in cands]
                )
                return cands[int(idx)][1]
            except Exception:
                return cands[0][1]

        time.sleep(0.1)

    return None


# ------------------ 主測試流程 ------------------
def run_tobacco_alcohol_tax_test(total_cases=100, seed=42):
    random.seed(seed)

    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 15)

    driver.delete_all_cookies()
    driver.get(URL)
    time.sleep(2)

    wait.until(EC.presence_of_element_located((By.ID, "select1")))

    DEGREE_REQUIRED = {2, 3, 5, 8}
    DEGREE_OPTIONAL = {4}

    mismatch = 0

    for i in range(total_cases):
        print(f"===== 迴圈第 {i+1} 次 =====")

        category = random.choice(["菸", "酒"])

        if category == "菸":
            sel1 = Select(wait.until(EC.element_to_be_clickable((By.ID, "select1"))))
            sel1.select_by_value("1")
            time.sleep(0.5)

            wait_select_ready(driver, wait, "select2", timeout=12)
            sel2_el = driver.find_element(By.ID, "select2")
            sel2 = Select(sel2_el)

            # 這裡關鍵：不要看 value，直接用 index（1..8）
            idxs = valid_option_indices(sel2)
            # 若 options 正好是 8 個（+placeholder），通常 idx 1..8 對應你定義的 1..8
            # 為避免選到超過 8 的奇怪項目，這裡先限制最多 8
            idxs = [x for x in idxs if 1 <= x <= 8]
            if not idxs:
                # 最後保底：就用原本可用 idxs（不限制），避免直接炸掉
                idxs = valid_option_indices(sel2)

            subcat = random.choice(idxs)
            sel2.select_by_index(subcat)
            trigger_change(driver, sel2_el)
            time.sleep(0.3)

            qty_el = wait.until(EC.presence_of_element_located((By.ID, "monthTax1")))
            qmin, qmax = get_int_constraints(qty_el, default_min=1, default_max=1_000_000)

            if subcat in (1, 5):
                base = random.randint(1, 10) * 1000
                quantity = min(max(base, qmin), qmax)
                quantity = (quantity // 1000) * 1000 if quantity >= 1000 else quantity
                if quantity <= 0:
                    quantity = 1000
            else:
                quantity = random.randint(max(1, qmin), min(100, qmax))

            local_tax = compute_tax("菸", subcat, quantity)

            safe_clear(driver, qty_el)
            human_type(qty_el, str(quantity), delay=0.1)

            anchor_id = "monthTax1"
            alcohol_content = None

        else:
            sel1 = Select(wait.until(EC.element_to_be_clickable((By.ID, "select1"))))
            sel1.select_by_value("2")
            time.sleep(0.5)

            wait_select_ready(driver, wait, "select3", timeout=12)
            sel3_el = driver.find_element(By.ID, "select3")
            sel3 = Select(sel3_el)

            # 同樣不要依賴 value，直接用 index（1..10）
            idxs = valid_option_indices(sel3)
            idxs = [x for x in idxs if 1 <= x <= 10]
            if not idxs:
                idxs = valid_option_indices(sel3)

            subcat = random.choice(idxs)
            sel3.select_by_index(subcat)
            trigger_change(driver, sel3_el)
            time.sleep(0.6)

            qty_el = wait.until(EC.presence_of_element_located((By.ID, "monthTax2")))
            qmin, qmax = get_int_constraints(qty_el, default_min=1, default_max=1_000_000)
            quantity = random.randint(max(1, qmin), min(100, qmax))

            # degree：依定義產生合法酒精度並確保填入
            alcohol_content = 1
            if subcat in DEGREE_REQUIRED:
                ok, alc = ensure_degree(driver, sel3_el, subcat, must_exist=True, retries=3)
                if ok and alc is not None:
                    alcohol_content = alc
                else:
                    # 保底：如果仍失敗，至少用合法範圍生成（避免亂填 1）
                    deg_el = find_degree_input(driver)
                    alcohol_content = choose_alcohol_content_for_subcat(subcat, deg_el)
                    print("⚠️ 需要酒精度的類別仍無法找到/填入 degree，這筆可能 mismatch")
            elif subcat in DEGREE_OPTIONAL:
                # subcat=4：>20（欄位存在就填）
                ok, alc = ensure_degree(driver, sel3_el, subcat, must_exist=False, retries=2)
                if ok and alc is not None:
                    alcohol_content = alc
                else:
                    alcohol_content = 21
            else:
                # 不需要 degree：若欄位存在就填 1 清殘值
                deg_el = find_degree_input(driver)
                if deg_el is not None and deg_el.is_enabled():
                    safe_clear(driver, deg_el)
                    human_type(deg_el, "1", delay=0.1)
                alcohol_content = 1

            local_tax = compute_tax("酒", subcat, quantity, alcohol_content)

            safe_clear(driver, qty_el)
            human_type(qty_el, str(quantity), delay=0.1)

            anchor_id = "monthTax2"

        # 計算
        btns = driver.find_elements(By.XPATH, "//button[@title='計算']")
        btns = [b for b in btns if b.is_displayed() and b.is_enabled()]
        if not btns:
            raise RuntimeError("找不到可點擊的『計算』按鈕")

        safe_click(driver, btns[0])
        time.sleep(0.8)

        online_tax = read_result_near_anchor(driver, anchor_id=anchor_id, timeout=10.0)

        if online_tax is None or local_tax != online_tax:
            mismatch += 1
            print(">>> 不一致參數發現!")
            print(f"Category: {category}")
            if category == "菸":
                print(f"子類別: {subcat}, 數量: {quantity}")
            else:
                print(f"子類別: {subcat}, 酒精成分: {alcohol_content}, 數量: {quantity}")
            print(f"本地計算稅額: {local_tax}, 線上稅額: {online_tax}")
        else:
            print("比對一致。")

        driver.refresh()
        time.sleep(2)
        wait.until(EC.presence_of_element_located((By.ID, "select1")))

    print(f"總測試次數: {total_cases}, 不一致次數: {mismatch}")
    driver.quit()


if __name__ == "__main__":
    run_tobacco_alcohol_tax_test(total_cases=100, seed=42)


===== 迴圈第 1 次 =====
比對一致。
===== 迴圈第 2 次 =====
比對一致。
===== 迴圈第 3 次 =====
>>> 不一致參數發現!
Category: 菸
子類別: 2, 數量: 76
本地計算稅額: 120840, 線上稅額: 0
===== 迴圈第 4 次 =====
比對一致。
===== 迴圈第 5 次 =====
比對一致。
===== 迴圈第 6 次 =====
比對一致。
===== 迴圈第 7 次 =====
比對一致。
===== 迴圈第 8 次 =====
比對一致。
===== 迴圈第 9 次 =====
比對一致。
===== 迴圈第 10 次 =====
比對一致。
===== 迴圈第 11 次 =====
比對一致。
===== 迴圈第 12 次 =====
比對一致。
===== 迴圈第 13 次 =====
比對一致。
===== 迴圈第 14 次 =====
比對一致。
===== 迴圈第 15 次 =====
比對一致。
===== 迴圈第 16 次 =====
比對一致。
===== 迴圈第 17 次 =====
比對一致。
===== 迴圈第 18 次 =====
比對一致。
===== 迴圈第 19 次 =====
比對一致。
===== 迴圈第 20 次 =====
比對一致。
===== 迴圈第 21 次 =====
比對一致。
===== 迴圈第 22 次 =====
比對一致。
===== 迴圈第 23 次 =====
比對一致。
===== 迴圈第 24 次 =====
比對一致。
===== 迴圈第 25 次 =====
比對一致。
===== 迴圈第 26 次 =====
比對一致。
===== 迴圈第 27 次 =====
比對一致。
===== 迴圈第 28 次 =====
比對一致。
===== 迴圈第 29 次 =====
比對一致。
===== 迴圈第 30 次 =====
比對一致。
===== 迴圈第 31 次 =====
比對一致。
===== 迴圈第 32 次 =====
比對一致。
===== 迴圈第 33 次 =====
比對一致。
===== 迴圈第 34 次 =====
比對一致。
===== 迴圈第 35 次 =====
比對一致。
===== 

### REVERIFY MISMATCHES PARAMS
Regarding the Tobacco and Alcohol Tax validation, we conducted 100 test runs with a 96% match rate. The 4 identified mismatches have been manually re-verified on the official portal and were found to be fully consistent with our local calculations.Technical Analysis of Discrepancies:Automation Lag (Zero-values): The cases where the portal returned a "0" tax amount (e.g., Test 3, 40, 77, and 84) were identified as synchronization issues within the Selenium automation script. In these instances, the script retrieved the value before the official web page had completed its asynchronous calculation.

The official portal: [https://www.etax.nat.gov.tw/etwmain/etw158w/75]